In [303]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [283]:
dataset_train = pd.read_csv("/content/train_u6lujuX_CVtuZ9i.csv")
dataset_test = pd.read_csv("/content/test_Y3wMUE5_7gLdaTN.csv")
dataset_test

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777,113.0,360.0,1.0,Urban
363,LP002975,Male,Yes,0,Graduate,No,4158,709,115.0,360.0,1.0,Urban
364,LP002980,Male,No,0,Graduate,No,3250,1993,126.0,360.0,NaN,Semiurban
365,LP002986,Male,Yes,0,Graduate,No,5000,2393,158.0,360.0,1.0,Rural


#Tien xu ly

In [284]:
dataset_train = dataset_train.iloc[:,2:9]
dataset_test = dataset_test.iloc[:,2:9]
dataset_test



,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount
0,Yes,0,Graduate,No,5720,0,110.0
1,Yes,1,Graduate,No,3076,1500,126.0
2,Yes,2,Graduate,No,5000,1800,208.0
3,Yes,2,Graduate,No,2340,2546,100.0
4,No,0,Not Graduate,No,3276,0,78.0
...,...,...,...,...,...,...,...
362,Yes,3+,Not Graduate,Yes,4009,1777,113.0
363,Yes,0,Graduate,No,4158,709,115.0
364,No,0,Graduate,No,3250,1993,126.0
365,Yes,0,Graduate,No,5000,2393,158.0


In [285]:
dataset_train = dataset_train.drop(dataset_train.columns[[4,5]], axis=1)
dataset_test = dataset_test.drop(dataset_test.columns[[4,5]], axis=1)

In [286]:
dataset_test

,Married,Dependents,Education,Self_Employed,LoanAmount
0,Yes,0,Graduate,No,110.0
1,Yes,1,Graduate,No,126.0
2,Yes,2,Graduate,No,208.0
3,Yes,2,Graduate,No,100.0
4,No,0,Not Graduate,No,78.0
...,...,...,...,...,...
362,Yes,3+,Not Graduate,Yes,113.0
363,Yes,0,Graduate,No,115.0
364,No,0,Graduate,No,126.0
365,Yes,0,Graduate,No,158.0


In [287]:
dataset_train['Married'].value_counts()
dataset_test['Married'].value_counts()

Yes    233
No     134
Name: Married, dtype: int64

In [43]:
dataset_train['Dependents'].value_counts()

0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64

In [44]:
dataset_train['Education'].value_counts()

Graduate        480
Not Graduate    134
Name: Education, dtype: int64

In [7]:
dataset_train['Self_Employed'].value_counts()

No     500
Yes     82
Name: Self_Employed, dtype: int64

In [288]:
dataset_train['Married'] = dataset_train['Married'].fillna("Yes")
dataset_test['Married'] = dataset_test['Married'].fillna("Yes")

In [289]:
dataset_train['Dependents'] = dataset_train['Dependents'].fillna(0)
dataset_test['Dependents'] = dataset_test['Dependents'].fillna(0)

In [290]:
dataset_train['Self_Employed'] = dataset_train['Self_Employed'].fillna("No")

dataset_test['Self_Employed'] = dataset_test['Self_Employed'].fillna("No")

In [291]:
dataset_train["LoanAmount"]=dataset_train["LoanAmount"].fillna(120)
dataset_test["LoanAmount"]=dataset_test["LoanAmount"].fillna(120)

In [130]:
dataset_train = dataset_train.dropna(how='any',axis=0)

In [268]:
X_train

array([[2.42100314, 0.        , 0.        , 0.        , 0.        ],
       [2.42100314, 0.        , 2.10090485, 0.99127589, 0.        ],
       [2.42100314, 0.        , 2.10090485, 0.        , 2.93971771],
       ...,
       [2.42100314, 0.        , 2.10090485, 0.99127589, 0.        ],
       [2.42100314, 0.        , 2.10090485, 1.98255178, 0.        ],
       [2.42100314, 0.        , 0.        , 0.        , 2.93971771]])

In [218]:
X_train[610]

array([1., 0., 1., 3., 0.])

In [292]:
def add_label(X): 
  for i in range(len(X)):
    print(i)
    if X[i] < 150:
      X[i] = 0
    elif X[i] < 300:
      X[i] = 1
    else:
      X[i] = 2

In [ ]:
add_label(dataset_train.LoanAmount)
add_label(dataset_test.LoanAmount)

In [294]:
le = LabelEncoder()
dataset_train['Married'] = le.fit_transform(dataset_train['Married'].astype(str))     
dataset_train['Self_Employed'] = le.fit_transform(dataset_train['Self_Employed'].astype(str))  
dataset_train['Dependents'] = le.fit_transform(dataset_train['Dependents'].astype(str))  
dataset_test['Married'] = le.fit_transform(dataset_test['Married'].astype(str))     
dataset_test['Self_Employed'] = le.fit_transform(dataset_test['Self_Employed'].astype(str))  
dataset_test['Dependents'] = le.fit_transform(dataset_test['Dependents'].astype(str)) 

In [295]:
X_train = dataset_train.iloc[:,0:-1]
Y_train = dataset_train.iloc[:,-1]
X_test = dataset_test.iloc[:,0:-1]
Y_test = dataset_test.iloc[:,-1]

In [296]:
ohe = ColumnTransformer([("one_hot_encoder", OneHotEncoder(), [2])], remainder = 'passthrough')
X_train = ohe.fit_transform(X_train)
X_test = ohe.fit_transform(X_test)

In [297]:
SC = StandardScaler(with_mean=False)
X_train = SC.fit_transform(X_train)
X_test =  SC.fit_transform(X_test)

#Su dung thuat toan SVM

In [298]:
classifier = SVC(kernel = "rbf")
classifier.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [300]:
train_scores = cross_val_score(classifier, X_train, Y_train, scoring='accuracy', cv = 10)
test_scores = cross_val_score(classifier, X_test, Y_test, scoring='accuracy', cv = 10)
print(train_scores.mean())
print(test_scores.mean())
cm = confusion_matrix(Y_train, classifier.predict(X_train))
print(cm)

0.6678476996298255
0.7002252252252253
[[390  17   0]
 [154  23   0]
 [ 25   5   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


# Su dung thuat toan Decision Free

In [301]:
classifier = DecisionTreeClassifier(criterion ="entropy")
classifier.fit(X_train, Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [302]:
train_scores = cross_val_score(classifier, X_train, Y_train, scoring='accuracy', cv = 10)
test_scores = cross_val_score(classifier, X_test, Y_test, scoring='accuracy', cv = 10)
print(train_scores.mean())
print(test_scores.mean())
cm = confusion_matrix(Y_train, classifier.predict(X_train))
print(cm)

0.6564251718667372
0.703003003003003
[[395  12   0]
 [156  21   0]
 [ 25   5   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


# Su Dung Thuat toan KNN

In [304]:
classifier = KNeighborsClassifier(n_neighbors= 3)
classifier.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [305]:
train_scores = cross_val_score(classifier, X_train, Y_train, scoring='accuracy', cv = 10)
test_scores = cross_val_score(classifier, X_test, Y_test, scoring='accuracy', cv = 10)
print(train_scores.mean())
print(test_scores.mean())
cm = confusion_matrix(Y_train, classifier.predict(X_train))
print(cm)

0.6533315705975674
0.6181681681681682
[[395  12   0]
 [157  20   0]
 [ 25   5   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
